# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y


In [1]:
# import libraries

import re
import joblib
import nltk
# nltk.download(['punkt', 'wordnet', 'stopwords'])
import pandas as pd
from sqlalchemy import create_engine
from classification_model.evaluation import print_cm, print_report
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
# load data from database
engine = create_engine('sqlite:///MessagesDisaster.db')
df = pd.read_sql_table('messages_disaster', con=engine)

cols = df.columns[4:]

X = df['message']
y = df.loc[:, cols]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", ' ', text.lower())
    words = word_tokenize(text=text)
    words = [word for word in words if word not in stopwords.words('english')]
    words = [WordNetLemmatizer().lemmatize(word, pos='v') for word in words]
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
rf_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                        ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced_subsample',
                                                                             random_state=42)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
rf_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                                                        class_weight='balanced_subsample',
                                                                        criterion='gini',
                                                                        max_depth=None,
                 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = rf_pipeline.predict(X_test)
print_report(y_test, y_pred)

The prediction for related is: 

              precision  recall  f1-score  support
0                  0.71    0.45      0.55  1873.00
1                  0.84    0.94      0.89  5934.00
2                  0.35    0.41      0.38    58.00
accuracy           0.82    0.82      0.82     0.82
macro avg          0.63    0.60      0.60  7865.00
weighted avg       0.81    0.82      0.80  7865.00
The prediction for request is: 

              precision  recall  f1-score  support
0                  0.91    0.98      0.94   6533.0
1                  0.82    0.52      0.64   1332.0
accuracy           0.90    0.90      0.90      0.9
macro avg          0.87    0.75      0.79   7865.0
weighted avg       0.89    0.90      0.89   7865.0
The prediction for offer is: 

              precision  recall  f1-score  support
0                  1.00     1.0      1.00   7829.0
1                  0.00     0.0      0.00     36.0
accuracy           1.00     1.0      1.00      1.0
macro avg          0.50     0.5     

In [8]:
print_cm(y_test, y_pred)

The confusion matrix for related is: 

Predicted     0     1     2
Observed                   
0          0.45  0.54  0.01
1          0.06  0.94  0.00
2          0.07  0.52  0.41
The confusion matrix for request is: 

Predicted     0     1
Observed             
0          0.98  0.02
1          0.48  0.52
The confusion matrix for offer is: 

Predicted    0    1
Observed           
0          1.0  0.0
1          1.0  0.0
The confusion matrix for aid_related is: 

Predicted     0     1
Observed             
0          0.82  0.18
1          0.30  0.70
The confusion matrix for medical_help is: 

Predicted     0     1
Observed             
0          1.00  0.00
1          0.93  0.07
The confusion matrix for medical_products is: 

Predicted     0     1
Observed             
0          1.00  0.00
1          0.95  0.05
The confusion matrix for search_and_rescue is: 

Predicted     0     1
Observed             
0          1.00  0.00
1          0.99  0.01
The confusion matrix for security is: 

P

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
rf_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                        ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced_subsample',
                                                                             random_state=42)))])

parameters = {
    'clf__estimator__n_estimators': [100, 200]
}

cv = GridSearchCV(rf_pipeline, param_grid=parameters, cv=3).fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_pred = cv.best_estimator_.predict(X_test)
print_report(y_test, y_pred)

The prediction for related is: 

              precision  recall  f1-score  support
0                  0.71    0.44      0.54  1873.00
1                  0.84    0.94      0.89  5934.00
2                  0.38    0.43      0.40    58.00
accuracy           0.82    0.82      0.82     0.82
macro avg          0.64    0.60      0.61  7865.00
weighted avg       0.81    0.82      0.80  7865.00
The prediction for request is: 

              precision  recall  f1-score  support
0                  0.91    0.98      0.94   6533.0
1                  0.83    0.53      0.64   1332.0
accuracy           0.90    0.90      0.90      0.9
macro avg          0.87    0.75      0.79   7865.0
weighted avg       0.90    0.90      0.89   7865.0
The prediction for offer is: 

              precision  recall  f1-score  support
0                  1.00     1.0      1.00   7829.0
1                  0.00     0.0      0.00     36.0
accuracy           1.00     1.0      1.00      1.0
macro avg          0.50     0.5     

In [11]:
print_cm(y_test, y_pred)

The confusion matrix for related is: 

Predicted     0     1     2
Observed                   
0          0.44  0.55  0.01
1          0.06  0.94  0.00
2          0.02  0.55  0.43
The confusion matrix for request is: 

Predicted     0     1
Observed             
0          0.98  0.02
1          0.47  0.53
The confusion matrix for offer is: 

Predicted    0    1
Observed           
0          1.0  0.0
1          1.0  0.0
The confusion matrix for aid_related is: 

Predicted     0     1
Observed             
0          0.82  0.18
1          0.29  0.71
The confusion matrix for medical_help is: 

Predicted     0     1
Observed             
0          1.00  0.00
1          0.94  0.06
The confusion matrix for medical_products is: 

Predicted     0     1
Observed             
0          1.00  0.00
1          0.96  0.04
The confusion matrix for search_and_rescue is: 

Predicted     0     1
Observed             
0          1.00  0.00
1          0.99  0.01
The confusion matrix for security is: 

P

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
gb_pipeline = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize, min_df=10)),
                        ('clf', MultiOutputClassifier(GradientBoostingClassifier(random_state=42)))])

params = {
    'clf__estimator__n_estimators': [100, 200],
    'clf__estimator__max_depth': [5, 7, 10]
}

gb_cv = GridSearchCV(gb_pipeline, param_grid=params, cv=3).fit(X_train, y_train)

y_pred = gb_cv.best_estimator_.predict(X_test)
print_report(y_test, y_pred)

ValueError: y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required.

In [ ]:
print_cm(y_test, y_pred)

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.